# Fine Tuning Transformer for MultiLabel Text Classification

###Installation

In [ ]:
!pip install -q transformers

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import ElectraTokenizer, ElectraModel, ElectraConfig
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
from tqdm import tqdm

<a id='section02'></a>
### Importing and Preprocessing Data



In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# workshop folder, e.g. 'acmlab/workshops/week3'
FOLDERNAME = 'Dataset'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1SGfUxG0kyYK011ICRcr-oyA-4S-p6H01/Dataset


In [ ]:
df = pd.read_csv('train.csv')
new_df = df[['text', 'is_humor']].copy()
new_df.head()

,text,is_humor
0,TENNESSEE: We're the best state. Nobody even c...,1
1,A man inserted an advertisement in the classif...,1
2,How many men does it take to open a can of bee...,1
3,Told my mom I hit 1200 Twitter followers. She ...,1
4,Roses are dead. Love is fake. Weddings are bas...,1


<a id='section03'></a>
### Dataset and Dataloader

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = ElectraTokenizer.from_pretrained("google/electra-large-discriminator")

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.is_humor
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (8000, 2)
TRAIN Dataset: (6400, 2)
TEST Dataset: (1600, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Fine-Tuning Neural Network

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of ELECTRA to get the final output for the model. 

class ELECTRAClass(torch.nn.Module):
    def __init__(self):
        super(ELECTRAClass, self).__init__()
        self.l1 = transformers.ElectraModel.from_pretrained("google/electra-large-discriminator")
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(1024, 1)
    
    def forward(self, ids, mask, token_type_ids):
        output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=True)['last_hidden_state'].mean(dim=1)
        #print(output_1.shape)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

weight_seed = 200
torch.manual_seed(weight_seed)
model = ELECTRAClass()
model.to(device)

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()

In [1]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

NameError: ignored

In [ ]:
writer = SummaryWriter(log_dir='/runs/N')

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
           # print(targets, outputs)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def perform_validation(epoch):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    targets = np.array(targets).reshape(-1, 1)
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    outputs_tensor = torch.FloatTensor(outputs)
    targets_tensor = torch.FloatTensor(targets)
    loss = loss_fn(outputs_tensor, targets_tensor)
    print()
    print(f"Val loss = {loss}")
    print(f"Val accuracy Score = {accuracy}")
    print(f"Val F1 Score (Micro) = {f1_score_micro}")
    print(f"Val F1 Score (Macro) = {f1_score_macro}")
    writer.add_scalar("Val loss", loss, epoch)
    writer.add_scalar("Val accuracy", accuracy, epoch)
    writer.add_scalar("Val F1 (micro)", f1_score_micro, epoch)
    writer.add_scalar("Val F2 (macro)", f1_score_macro, epoch)

In [ ]:
newest_df = test_dataset.copy(deep=True)

In [ ]:
def validation_and_save(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    fin_outputs = (np.array(fin_outputs) >= 0.5).astype(int)
    newest_df.insert(loc=epoch+2, column=f'predictions-{epoch+1}', value=fin_outputs)
    newest_df.to_csv(f'predictions-{weight_seed}-epoch{epoch+1}.csv')



In [ ]:
def train(epoch):
    model.train()
    losses = []
    for _,data in tqdm(enumerate(training_loader, 0), total=len(training_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs.squeeze(), targets.squeeze())
        losses.append(loss.detach())
        if _ % 25 == 0:
            writer.add_scalar("Loss/train", losses[-1], float(_ % 5000) + float(epoch))
        if len(losses) == 50:
            print(f'Epoch: {epoch}, Loss:  {torch.mean(torch.stack(losses))}')
            perform_validation(epoch)
            print()
            losses.clear()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
EPOCHS = 3                      # changed from 5
for epoch in range(EPOCHS):
    train(epoch)
    perform_validation(epoch)

  0%|          | 0/400 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
 12%|█▏        | 49/400 [01:43<11:52,  2.03s/it]

Epoch: 0, Loss:  0.6145930886268616

Val loss = 0.5574231147766113
Val accuracy Score = 0.82875
Val F1 Score (Micro) = 0.82875
Val F1 Score (Macro) = 0.7961309523809523



 25%|██▍       | 99/400 [04:42<10:07,  2.02s/it]

Epoch: 0, Loss:  0.3226768374443054

Val loss = 0.49711886048316956
Val accuracy Score = 0.914375
Val F1 Score (Micro) = 0.914375
Val F1 Score (Macro) = 0.9084511836401596



 37%|███▋      | 149/400 [07:39<08:20,  1.99s/it]

...

Epoch: 2, Loss:  0.03075265884399414

Val loss = 0.4769163131713867
Val accuracy Score = 0.949375
Val F1 Score (Micro) = 0.949375
Val F1 Score (Macro) = 0.9464984339645774



100%|█████████▉| 399/400 [22:25<00:02,  2.02s/it]

Epoch: 2, Loss:  0.03763146325945854

Val loss = 0.47963282465934753
Val accuracy Score = 0.9525
Val F1 Score (Micro) = 0.9525
Val F1 Score (Macro) = 0.9488391192250474



100%|██████████| 400/400 [23:45<00:00,  3.56s/it]



Val loss = 0.4779323935508728
Val accuracy Score = 0.955
Val F1 Score (Micro) = 0.955
Val F1 Score (Macro) = 0.9516355740504703


In [ ]:
torch.save(model.state_dict(), "300-jan-30-humor-classification.pt")